# Using cross-location and cross-domain corpora for emotion classification of tweets about museum experiences 


## Coding workflow

## Part 1: Data collection

### Using Tweepy package, collecting data from Twitter API with New York City museums' mentions (@) and hashtags (#)

In [ ]:
# Importing necessary packages

import tweepy
from tweepy.auth import OAuthHandler
import json 
import pandas
import nltk
import from sklearn.metrics import confusion_matrix
import itertools
import numpy as np
import matplotlib.pyplot as plt
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from nltk import word_tokenize          
from nltk.stem import WordNetLemmatizer 
from sklearn.svm import LinearSVC
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score, classification_report
from sklearn.naive_bayes import MultinomialNB

# Accessing Twitter API using information in a created Twitter Apps which provides access tokens: https://apps.twitter.com/

consumer_key = 'xxxxxxxxxxxx' 
consumer_secret = 'xxxxxxxxxxxxx' 
access_token = 'xxxxxxxxxxxx' 
access_secret = 'xxxxxxxxxxxx' 
auth = OAuthHandler(consumer_key, consumer_secret) 
auth.set_access_token(access_token, access_secret) 
api = tweepy.API(auth, parser=tweepy.parsers.JSONParser()) # Using JSON Parser to read the Twitter formatting

# Reference link: http://www.tweepy.org/
# Reference link: https://stackoverflow.com/

### Creating lists with 130 New York City museums' Twitter handles and hashtags

In [ ]:
handles_list = ['@iAliceAusten','@americanacad','@FolkArtMuseum','@EllisIsland','@AMNH','@ANSCoins','@Visual_ArtsAS','@AnneFrankCenter',
                '@AsiaSociety','@NYCAudubon','@Bartow_Pell','@BowneHouse1','@BronxHistory','@BronxMuseum','@BrooklynKids','@brooklynhistory',
                '@brooklynmuseum','@ChelseaArtMus','@cmomNYC','@CMAinNYC','@met_cloisters','@coneyislandusa','@cooperhewitt','@DaheshShop',
                '@DiaFoundation','@DiscoveryTS','@drawingcenter','@DyckmanFarm','@elmuseo','@frauncestavern','@frickcollection',
                '@GI_NewYork','@gracie_mansion','@HSAmuseum','@HolocaustTolCtr','@MuseumBarge','@ICPhotog','@ICPlibrary',
                '@ICPStore','@IntrepidMuseum','@NoguchiMuseum','@IAMMuseumNYC','@TibetanMuseum','@TheJewishMuseum',
                '@nycwax','@MerchantsHouse','@metmuseum','@MCINY','@MorganLibrary','@MorrisJumel','@MuseumMVH','@MASNYC',
                '@FinanceMuseum','@MADMuseum','@mocanyc','@MoCADA','@MJHnews','@MuseumModernArt','@MuseumofCityNY',
                '@MovingImageNYC','@NatlAcademy','@AmerIndianNYC','@Sept11Memorial','@NeueGalerieNY','@newmuseum','@FDNYMuseum',
                '@nycpolicemuseum','@nysci','@NYHistory','@NYTransitMuseum','@OSHBklyn','@MoMAPS1','@paleycenter',
                '@QueensMuseum','@RubinMuseum','@ScanHouse','@SchomburgCenter','@SVAgalleries','@SkyMuseum','@Guggenheim',
                '@SonySquareNYC','@SeaportMuseum','@SIMuseum','@studiomuseum','@tenementmuseum','@NatlJazzMuseum','@museumatFIT',
                '@TRBirthplaceNPS','@UkrMuseum','@WaveHill','@whitneymuseum','@WyckoffMuseum','@lbinyc','@AmericanSephard',
                '@yivoinstitute','@AJHSNYC','@YUMuseum','@cjewishhistory']

In [ ]:
hashtag_list = ['#iAliceAusten','#americanacad','#FolkArtMuseum','#EllisIsland','#AMNH','#ANSCoins','#Visual_ArtsAS','#AnneFrankCenter',
                '#AsiaSociety','#NYCAudubon','#Bartow_Pell','#BowneHouse1','#BronxHistory','#BronxMuseum','#BrooklynKids','#brooklynhistory',
                '#brooklynmuseum','#ChelseaArtMus','#cmomNYC','#CMAinNYC','#met_cloisters','#coneyislandusa','#cooperhewitt','#DaheshShop',
                '#DiaFoundation','#DiscoveryTS','#drawingcenter','#DyckmanFarm','#elmuseo','#frauncestavern','#frickcollection',
                '#GI_NewYork','#gracie_mansion','#HSAmuseum','#HolocaustTolCtr','#MuseumBarge','#ICPhotog','#ICPlibrary',
                '#ICPStore','#IntrepidMuseum','#NoguchiMuseum','#IAMMuseumNYC','#TibetanMuseum','#TheJewishMuseum',
                '#nycwax','#MerchantsHouse','#metmuseum','#MCINY','#MorganLibrary','#MorrisJumel','#MuseumMVH','#MASNYC',
                '#FinanceMuseum','#MADMuseum','#mocanyc','#MoCADA','#MJHnews','#MuseumModernArt','#MuseumofCityNY',
                '#MovingImageNYC','#NatlAcademy','#AmerIndianNYC','#Sept11Memorial','#NeueGalerieNY','#newmuseum','#FDNYMuseum',
                '#nycpolicemuseum','#nysci','#NYHistory','#NYTransitMuseum','#OSHBklyn','#MoMAPS1','#paleycenter',
                '#QueensMuseum','#RubinMuseum','#ScanHouse','#SchomburgCenter','#SVAgalleries','#SkyMuseum','#Guggenheim',
                '#SonySquareNYC','#SeaportMuseum','#SIMuseum','#studiomuseum','#tenementmuseum','#NatlJazzMuseum','#museumatFIT',
                '#TRBirthplaceNPS','#UkrMuseum','#WaveHill','#whitneymuseum','#WyckoffMuseum','#lbinyc','#AmericanSephard',
                '#yivoinstitute','#AJHSNYC','#YUMuseum','#cjewishhistory']

### Downloading data

In [ ]:
list_for_df_status = [] # Creating empty list for status data 
for parameter in handles_list: # Iterating through a list of 130 museum handles (for hashtags, use hashtag_list) 
    tweets = api.search(parameter) # Specifying query parameter which is a Twitter handle or hashtag
    text = json.dumps(tweets) # Loading tweet content 
    json_file = json.loads(text) 
    for i in json_file['statuses']: # Extracting status of tweets 
        status = i['text']  
        list_for_df_status.append(status) 

In [ ]:
# Optional: saving results to a .csv filee

data = pandas.DataFrame(list_for_df_status, columns=['status'])
data.to_csv('Sample.csv')

## Part 2: Analysis

### Importing data

In [ ]:
# Importing necessary packages

import nltk
import csv

In [ ]:
# Importing text file for analysis and creating a list

text = []
with open('Train_text.csv', encoding="latin-1") as f:
    reader = csv.reader(f, delimiter='\t')
    for i in reader:
        text.append(i[0])

# Reference link: https://stackoverflow.com/

In [ ]:
# Importing file with associated labels

labels = []
with open('Train_labels.csv', encoding='utf-8') as f:
    reader = csv.reader(f, delimiter='\t')
    for i in reader:
        labels.append(i[0])

In [ ]:
# Creating a test file with text

text_list = []
with open('Test_text.csv', encoding='latin-1') as f:
    reader = csv.reader(f, delimiter='\t')
    for i in reader:
        text_list.append(i[0])

In [ ]:
# Importing file with associated labels for test

labels_test = []
with open('Test_labels.csv', encoding='utf-8') as f:
    reader = csv.reader(f, delimiter='\t')
    for i in reader:
        labels_test.append(i[0])

### Creating text pre-processing, defining metrics

In [ ]:
class LemmaTokenizer(object):
    def __init__(self):
        self.wnl = WordNetLemmatizer()
    def __call__(self, doc):
        return [self.wnl.lemmatize(t) for t in word_tokenize(doc)]

# Reference link: http://scikit-learn.org/stable/modules/feature_extraction.html

In [ ]:
def metrics(y_test, y_predicted):  
    precision = precision_score(y_test, y_predicted, pos_label=None,
                                     average='weighted')             
    recall = recall_score(y_test, y_predicted, pos_label=None,
                              average='weighted')
    f1 = f1_score(y_test, y_predicted, pos_label=None, average='weighted')
    accuracy = accuracy_score(y_test, y_predicted)
    return accuracy, precision, recall, f1

# Reference link: https://github.com/hundredblocks/concrete_NLP_tutorial/blob/master/NLP_notebook.ipynb

### Creating term frequency-inverse document frequency matrix for train and test data

In [ ]:
def tfidf(data):
    tfidf_vectorizer = TfidfVectorizer(tokenizer=LemmaTokenizer())
    train = tfidf_vectorizer.fit_transform(data)
    return train, tfidf_vectorizer

X_train, tfidf_vectorizer = tfidf(text)
X_test = tfidf_vectorizer.transform(text_list)

# Reference link: https://github.com/hundredblocks/concrete_NLP_tutorial/blob/master/NLP_notebook.ipynb

### Running classifier: Linear SVM

In [ ]:
clf = LinearSVC(random_state=0)
clf.fit(X_train, labels_ready)
y_predicted = clf.predict(X_test)

y_test = labels_ready_test
accuracy, precision, recall, f1 = metrics(y_test, y_predicted)
print("accuracy = %.2f, precision = %.2f, recall = %.2f, f1 = %.2f" % (accuracy, precision, 
                                                                       recall, f1))

# Reference link: https://github.com/hundredblocks/concrete_NLP_tutorial/blob/master/NLP_notebook.ipynb

### Plotting confusion matrix

In [ ]:
class_names = ['angry', 'disgust','happy','sad','surprise']

def plot_confusion_matrix(cm, classes,
                          normalize=False,
                          title='Confusion matrix',
                          cmap=plt.cm.Blues):

    print(cm)

    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=45)
    plt.yticks(tick_marks, classes)

    fmt = '.2f' if normalize else 'd'
    thresh = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, format(cm[i, j], fmt),
                 horizontalalignment="center",
                 color="white" if cm[i, j] > thresh else "black")

    plt.tight_layout()
    plt.ylabel('True label')
    plt.xlabel('Predicted label')

cnf_matrix = confusion_matrix(y_test, y_predicted)
np.set_printoptions(precision=2)

plt.figure()
plot_confusion_matrix(cnf_matrix, classes=class_names,
                      title='Name')

plt.show()

# Reference link: http://scikit-learn.org/stable/auto_examples/model_selection/plot_confusion_matrix.html#sphx-glr-auto-examples-model-selection-plot-confusion-matrix-py

### Running classifier: Multinomial Naive Bayes

In [ ]:
clf = MultinomialNB()
clf.fit(X_train, labels_ready)

y_test = labels_ready_test
y_predicted = clf.predict(X_test)

accuracy, precision, recall, f1 = metrics(y_test, y_predicted)
print("accuracy = %.2f, precision = %.2f, recall = %.2f, f1 = %.2f" % (accuracy, precision, 
                                                                       recall, f1))

### Plotting confusion matrix

In [ ]:
cnf_matrix = confusion_matrix(y_test, y_predicted)
np.set_printoptions(precision=2)

plt.figure()
plot_confusion_matrix(cnf_matrix, classes=class_names,
                      title='Name')

plt.show()

# Reference link: http://scikit-learn.org/stable/auto_examples/model_selection/plot_confusion_matrix.html#sphx-glr-auto-examples-model-selection-plot-confusion-matrix-py

## References:
### 1. Tweepy package: http://www.tweepy.org/
### 2. Emmanuel Ameisen. How to solve 90% of NLP problems: a step-by-step guide.
###  https://github.com/hundredblocks/concrete_NLP_tutorial/blob/master/NLP_notebook.ipynb
### 3. Sklearn package:
### http://scikit-learn.org/stable/auto_examples/model_selection/plot_confusion_matrix.html#sphx-glr-auto-examples-model-selection-plot-confusion-matrix-py
### http://scikit-learn.org/stable/modules/feature_extraction.html
### 4. Stackoverflow: https://stackoverflow.com/
